In [43]:
import pandas as pd
pat_id_offset = pd.read_csv('../data/intermediate/id_offset.csv')
test_lab = pd.read_csv('../data/intermediate/test_lab_data.csv', header=None)

In [45]:
columns = ['patientunitstayid', 'labname', 'labresult', 'labresultoffset']
test_lab.columns=columns

In [46]:
test_lab.set_index(['patientunitstayid','labresultoffset', 'labname'], inplace=True)
test_lab = test_lab.unstack(level=2)

In [47]:
test_lab.columns = test_lab.columns.droplevel()

In [71]:
test_lab.reset_index(level=1, inplace=True)

In [49]:
pat_id_offset.set_index('patientunitstayid', inplace=True)

In [72]:
test_lab = test_lab.merge(pat_id_offset, left_index=True, right_index=True)

In [73]:
test_lab['time_to_detect'] = test_lab['earlier_time'] - test_lab['labresultoffset']

In [52]:
pd.option_context("mode.chained_assignment", None)
test_lab = test_lab[test_lab['time_to_detect']/60>=12]
test_lab.loc[test_lab['time_to_detect']/60<=24, 'positive_window'] = 1

/Users/zhannahakhverdyan/anaconda3/envs/predict_dialisys_need/lib/python3.8/site-packages/pandas/core/indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/zhannahakhverdyan/anaconda3/envs/predict_dialisys_need/lib/python3.8/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [74]:
test_lab.fillna(0, inplace=True)

In [75]:
test_lab.reset_index(inplace=True)
test_lab['time_to_detect'] = pd.to_timedelta(test_lab['time_to_detect'], unit='minute')
test_lab.set_index('time_to_detect', inplace=True)

In [77]:
test_lab.reset_index(inplace=True)
min_df = test_lab.groupby('patientunitstayid').apply(lambda x: x.rolling('1d', min_periods=2).agg({'creatinine':'min', 'BUN':'min', 'WBC x 1000':'min'}))
min_df

,creatinine,BUN,WBC x 1000
time_to_detect,,,
1 days 01:10:00,NaN,NaN,NaN
0 days 14:53:00,1.95,26.0,9.8
1 days 02:21:00,NaN,NaN,NaN
2 days 00:48:00,NaN,NaN,NaN
1 days 00:50:00,1.40,26.0,47.7
0 days 22:55:00,1.40,26.0,47.7
1 days 00:45:00,NaN,NaN,NaN


In [78]:
creat_df = test_lab.groupby('patientunitstayid')[['creatinine', 'BUN', 'WBC x 1000']].apply(lambda x: x.rolling('2d', min_periods=2).agg(['min', 'median', 'max'])) 
creat_df

creatinine               BUN              WBC x 1000         \
                       min median  max   min median   max        min median   
time_to_detect                                                                
1 days 01:10:00        NaN    NaN  NaN   NaN    NaN   NaN        NaN    NaN   
0 days 14:53:00       1.95  2.125  2.3  26.0   26.5  27.0        9.8  12.25   
1 days 02:21:00        NaN    NaN  NaN   NaN    NaN   NaN        NaN    NaN   
2 days 00:48:00        NaN    NaN  NaN   NaN    NaN   NaN        NaN    NaN   
1 days 00:50:00       1.40  1.400  1.4  26.0   28.5  31.0       47.7  47.95   
0 days 22:55:00       1.40  1.400  1.5  26.0   26.0  31.0       47.7  48.20   
1 days 00:45:00        NaN    NaN  NaN   NaN    NaN   NaN        NaN    NaN   

                       
                  max  
time_to_detect         
1 days 01:10:00   NaN  
0 days 14:53:00  14.7  
1 days 02:21:00   NaN  
2 days 00:48:00   NaN  
1 days 00:50:00  48.2  
0 days 22:55:00  65.9  
1 days 00:45:00   NaN

In [80]:
#creat_df.columns = creat_df.columns.map('_'.join)
creat_df = pd.concat([creat_df, test_lab[['patientunitstayid', 'labresultoffset']]], axis=1, ignore_index=False)
creat_df

,creatinine_min,creatinine_median,creatinine_max,BUN_min,BUN_median,BUN_max,WBC x 1000_min,WBC x 1000_median,WBC x 1000_max,patientunitstayid,labresultoffset
time_to_detect,,,,,,,,,,,
1 days 01:10:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,141168,516
0 days 14:53:00,1.95,2.125,2.3,26.0,26.5,27.0,9.8,12.25,14.7,141168,1133
1 days 02:21:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,141203,-1580
2 days 00:48:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,141227,-1566
1 days 00:50:00,1.40,1.400,1.4,26.0,28.5,31.0,47.7,47.95,48.2,141227,-128
0 days 22:55:00,1.40,1.400,1.5,26.0,26.0,31.0,47.7,48.20,65.9,141227,-13
1 days 00:45:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,141296,-1827


In [81]:
test_lab

,patientunitstayid,labresultoffset,BUN,WBC x 1000,creatinine,earlier_time,positive_window
time_to_detect,,,,,,,
1 days 01:10:00,141168,516,26.0,9.8,1.95,2026.0,0.0
0 days 14:53:00,141168,1133,27.0,14.7,2.30,2026.0,1.0
1 days 02:21:00,141203,-1580,10.0,10.2,0.33,1.0,0.0
2 days 00:48:00,141227,-1566,31.0,48.2,1.40,1362.0,0.0
1 days 00:50:00,141227,-128,26.0,47.7,1.40,1362.0,0.0
0 days 22:55:00,141227,-13,26.0,65.9,1.50,1362.0,1.0
1 days 00:45:00,141296,-1827,31.0,14.6,4.00,-342.0,0.0
